In [17]:
from nltk.stem import SnowballStemmer
import gensim as gensim
from gensim.models.word2vec import Word2Vec
from math import *
import numpy as np
from sklearn.manifold import TSNE
from string import *
import gensim.models
from sklearn import linear_model
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
from sklearn import neural_network
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import io

%matplotlib inline

In [9]:
#the pre-tarined weights:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

In [10]:
#tokenization:

def get_non_alphanumeric_chars(input_text):
    """
        Returns the non alphanumeric unique characters from a corpus.
        Input: 
            * input_text:   string
        Output:
            * non_alphanumeric_unique: list of non alphanumeric characters
    """     
    ###Start code here
    
    #STEP1 set character_list variable to a list of characters in the input_text 
    character_list = list(input_text)
    
    #STEP2 store all non_alphanumeric characters from the character_list!
    non_alphanumeric = []
    for s in character_list:
        if (s.isalnum()==False):
            non_alphanumeric.append(s)
            #character_list.delete(s)
    
    #STEP3 keep only the unique characters (hint: unique_values = list(set(duplicated_values))
    non_alphanumeric_unique = list(set(non_alphanumeric))
    
    ###End code here 
    
    return non_alphanumeric_unique

def tokenizer(input_text):
    """
        Transforms strings to tokens. The tokens should appear in the same order as they are in the string.
        Input: 
            * input_text:   string
        Output:
            * tokens: list of tokens
    """    

    
    ###Start code here
    #STEP0 extract non alphanumeric characters! Use your function above!
    non_alphanumeric_unique = get_non_alphanumeric_chars(input_text)
    
    #STEP1 replace all non alphanumeric characters to ' ' whitespace.
    replaced_input = input_text
    for s in replaced_input:
        if (s in non_alphanumeric_unique):
            replaced_input=replaced_input.replace(s," ")
    
    #STEP2 split the input_text on all the non_alphanumeric_unique characters and store then in the tokens variable
    tokens = replaced_input.split()
    
    ###End code here    
    
    
    return tokens

In [11]:
#train corpus:
corpusfile=open('./trainemoji.txt', 'r', encoding='utf-8')
lines=corpusfile.readlines()
corpusfile.close()

In [12]:
#example for a tokenized sentence:
tokenizer(lines[3][1:].lower())

['we',
 'have',
 'to',
 'hurt',
 'in',
 'order',
 'to',
 'know',
 'fall',
 'in',
 'order',
 'to',
 'grow',
 'lose',
 'in',
 'order',
 'to',
 'gain',
 'because',
 'all',
 'of',
 'life',
 's',
 'lessons',
 'are',
 'taught',
 'through',
 'pain']

In [13]:
#vectorization of sentences:
def vectorize(filename,model):
    file=open(filename, "r", encoding="utf-8")
    lines=file.readlines()
    file.close()
    VECTORS=np.empty([len(lines),300])
    for IDX in range(len(lines)):
        W=np.zeros(300)
        N=int(0)
        for idx in range(len(tokenizer(lines[IDX][1:].lower()))):
            s=tokenizer(lines[IDX][1:].lower())[idx]
            if s in model:
                W+=np.array(model.get_vector(s))
                N+=1
        W=W/N
        for i in range(len(W)):
            VECTORS[IDX][i]=W[i]
    return (VECTORS)

In [14]:
#labels of senteces:
def labelize(filename, model):
    with io.open(filename, 'r', encoding='utf-8') as corpusfile:
        lines=corpusfile.readlines()
    corpusfile.close()
    lineslabels=np.empty(len(lines))
    for IDX in range(len(lines)):
        lineslabels[IDX]=float(str(tokenizer(lines[IDX].lower())[0]))
        
    return lineslabels

In [21]:
#First, let's try using random forests.
classify=RandomForestClassifier(n_estimators=50, n_jobs=20, max_features=20, criterion='entropy')
print(classify.fit(vectorize('./trainemoji.txt',model), labelize('./trainemoji.txt',model)))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=20, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=20,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [22]:
testvectors=vectorize('./testemoji.txt',model)
testlabels=labelize('./testemoji.txt',model)

In [24]:
print('Original:\n')
print(testlabels)
print('Fitted:\n')
print(classify.predict(testvectors))
acc = cross_val_score(classify, testvectors, testlabels, cv=5)
print('Accuracy: '+str((np.average(acc))*100)+'% +/- '+str((np.std(acc))*100)+'%\n')

Original:

[2. 2. 3. 0. 0. 2. 0. 3. 1. 0. 2. 1. 2. 1. 0. 2. 0. 1. 3. 2. 2. 2. 3. 0.
 1. 1. 0. 0. 3. 0. 0. 3. 3. 1. 3. 0. 0. 1. 3. 1. 2. 1. 1. 0. 2. 1.]
Fitted:

[1. 1. 0. 0. 0. 2. 0. 3. 0. 0. 0. 1. 2. 1. 0. 2. 0. 1. 1. 2. 1. 2. 1. 0.
 2. 1. 0. 0. 1. 0. 0. 1. 3. 0. 3. 0. 0. 1. 2. 1. 1. 0. 0. 0. 2. 1.]
Accuracy: 65.86147186147187% +/- 5.95766628787512%



In [25]:
#Thi should work better.

In [26]:
#Let's try classification with fully connected neura networks instead.
network=MLPClassifier(hidden_layer_sizes=(10,20,8,4), solver='sgd', alpha=0.0001, activation='tanh', batch_size=20, max_iter=1000, random_state=1)
print(network.fit(vectorize('./trainemoji.txt',model), labelize('./trainemoji.txt',model)))

MLPClassifier(activation='tanh', alpha=0.0001, batch_size=20, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 20, 8, 4), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


In [28]:
print('Original:\n')
print(testlabels)
print('Fitted:\n')
print(network.predict(testvectors))
accn = cross_val_score(network, testvectors, testlabels, cv=5)
print('Accuracy: '+str((np.average(accn))*100)+'% +/- '+str((np.std(accn))*100)+'%\n')

Original:

[2. 2. 3. 0. 0. 2. 0. 3. 1. 0. 2. 1. 2. 1. 0. 2. 0. 1. 3. 2. 2. 2. 3. 0.
 1. 1. 0. 0. 3. 0. 0. 3. 3. 1. 3. 0. 0. 1. 3. 1. 2. 1. 1. 0. 2. 1.]
Fitted:

[2. 0. 2. 0. 0. 2. 0. 3. 1. 0. 2. 1. 2. 2. 0. 2. 0. 1. 3. 2. 1. 2. 1. 0.
 3. 1. 0. 0. 3. 0. 0. 3. 3. 0. 3. 0. 0. 1. 3. 1. 2. 1. 1. 0. 2. 1.]


/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 76.7994227994228% +/- 5.370357424484229%



/usr/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [29]:
import emoji
print(emoji.emojize('This is number 0: :heart_suit:'))
print(emoji.emojize('This is number 1: :disappointed_face:'))
print(emoji.emojize('This is number 2: :airplane_departure:'))
print(emoji.emojize('This is number 3: :fork_and_knife:'))

This is number 0: ♥
This is number 1: 😞
This is number 2: 🛫
This is number 3: 🍴


In [40]:
#Let's try to substitute communication with emojis, and make our parents proud.
linesfile=open('./testemoji.txt', 'r', encoding='utf-8')
lines=linesfile.readlines()
linesfile.close()
emojinum=int(0)
emojistring=' '
for i in range(len(network.predict(testvectors))):
    emojinum=network.predict(testvectors)[i]
    if emojinum==0:
        emojistring=':heart_suit:'
    elif emojinum==1:
        emojistring=':disappointed_face:'
    elif emojinum==2:
        emojistring=':airplane_departure:'
    elif emojinum==3:
        emojistring=':fork_and_knife:'
    print(emoji.emojize(emojistring)+' '+lines[i])

🛫 2 The first condition of understanding a foreign country is to smell it.

♥ 2 Those who know nothing of foreign languages know nothing of their own.

🛫 3 Ask not what you can do for your country. Ask what's for lunch.

♥ 0 You are, and always have been, my dream.

♥ 0 You're my dream lover, and I couldn't picture someone better suited for me than you.

🛫 2 Jet lag is for amateurs.

♥ 0 My love and adoration for you only grows stronger with time. Much, much stronger.

🍴 3 The odds of going to the store for a loaf of bread and coming out with only a loaf of bread are three billion to one.

😞 1 You still mean everything to me, but you're just not worth the fight anymore.

♥ 0 Seeing you was love at first sight. Seeing you changed my life forever.

🛫 2 Let your memory be your travel bag.

😞 1 Sometimes we do have to be selfless not selfish and think of what's going to be the best for the ones we love.

🛫 2 Every dreamer knows that it is entirely possible to be homesick for a place you've